In [2]:
!pip install hyperopt
!pip install datasets
!pip install tensorflow
!pip install keras_tuner
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.6 MB/s eta 0:00:00


In [3]:
dataset = load_dataset('imdb', split='train', download_mode="force_redownload")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
import pandas as pd
import numpy as np
import regex as re
import string
from nltk import download as nltk_download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk_download('stopwords')
nltk_download('punkt')
nltk_download('wordnet')

slang_dict = {
    'lol': 'laugh out loud',
    'brb': 'be right back',
    'idk': 'I don\'t know',
    'imo': 'in my opinion',
    'imho': 'in my humble opinion',
    'btw': 'by the way',
    'b4': 'before',
    'fyi': 'for your information',
    'irl': 'in real life',
    'thx': 'thanks',
    'smh': 'shaking my head',
    'afaik': 'as far as I know',
    'asap': 'as soon as possible',
    'atm': 'at the moment',
    'bfn': 'bye for now',
    'cya': 'see you',
    'ftw': 'for the win',
    'gg': 'good game',
    'gr8': 'great',
    'hbu': 'how about you',
    'jk': 'just kidding',
    'lmao': 'laughing my ass off',
    'lmk': 'let me know',
    'nvm': 'never mind',
    'omg': 'oh my god',
    'ppl': 'people',
    'rofl': 'rolling on the floor laughing',
    'tbh': 'to be honest',
    'tgif': 'thank goodness it\'s Friday',
    'ttyl': 'talk to you later',
    'wtf': 'what the f***',
    'yw': 'you\'re welcome',
    'np': 'no problem',
    'plz': 'please',
    'u': 'you',
    'r': 'are',
    '2': 'to/too',
    '4': 'for',
    'b/c': 'because',
    'cuz': 'because',
    'yr': 'year',
    'yolo': 'you only live once',
    'tmi': 'too much information',
    'bff': 'best friends forever',
    'fb': 'Facebook',
    'dm': 'direct message',
    'mtfbwy': 'may the force be with you',
    'tldr': 'too long; didn\'t read',
    'fomo': 'fear of missing out',
    'ftl': 'for the loss',
    'wbu': 'what about you',
    'hmu': 'hit me up',
    'idc': 'I don\'t care',
    'iso': 'in search of',
    'jic': 'just in case',
    'qotd': 'quote of the day',
    'rn': 'right now',
    'sfw': 'safe for work',
    'nsfw': 'not safe for work',
    'icymi': 'in case you missed it',
    'tl;dr': 'too long; didn\'t read'
}

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

url_re = re.compile(r'http\S+')
html_re = re.compile(r'<.*?>')


text = dataset['text']
label = dataset['label']

tokenized = []

for i in range(len(text)):
  text[i] = re.sub(html_re, ' ', text[i]) #Remove all apparent HTML tags.
  text[i] = re.sub(url_re, '', text[i]) #Remove all URLs.
  text[i] = text[i].lower()
  text[i] = text[i].translate(str.maketrans('', '', string.punctuation))

  tokens = word_tokenize(text[i])
  tokens = [word for word in tokens if word not in stop_words]
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  tokens = [slang_dict.get(token, token) for token in tokens]

  tokenized.append(' '.join(tokens))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
#Get size of vocabulary.
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(tokenized)
vocabulary_size = len(vectorizer.get_feature_names_out())
print(vocabulary_size)

102343


In [6]:
max_length = 0
for l in tokenized:
  if len(l) >= max_length:
    max_length = len(l)
print(max_length)

9220


In [7]:
df = pd.DataFrame({'Reviews': tokenized, 'Label': label})
print(df)

                                                 Reviews  Label
0      rented curiousyellow video store controversy s...      0
1      curious yellow risible pretentious steaming pi...      0
2      avoid making type film future film interesting...      0
3      film probably inspired godard masculin féminin...      0
4      oh brotherafter hearing ridiculous film umptee...      0
...                                                  ...    ...
24995  hit time better categorised australian cult fi...      1
24996  love movie like another time try explain virtu...      1
24997  film sequel barry mckenzie hold two greatest c...      1
24998  adventure barry mckenzie started life satirica...      1
24999  story center around barry mckenzie must go eng...      1

[25000 rows x 2 columns]


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Reviews'], df['Label'], train_size = 0.8, random_state = 42)

In [9]:
#Compiling NN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

max_length = 150
num_classes = 1 #Binary classification

tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(' '.join(tokenized))

train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded_sequences = pad_sequences(train_sequences, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded_sequences = pad_sequences(test_sequences, maxlen=max_length)


def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=vocabulary_size, output_dim=hp.Int('embedding_dim', min_value=128, max_value=256, step=32), input_length=max_length))
    model.add(LSTM(hp.Int('units', min_value=128, max_value=256, step=32)))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.3, step=0.1)))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer=Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint
import keras_tuner

batch_size = 32
epochs = 30

checkpoint = ModelCheckpoint('model_checkpoint.h5', save_best_only=True, monitor='val_loss', mode='min')

tuner = keras_tuner.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        executions_per_trial=2,
                        directory='/content/',
                        project_name='hparam_tuning')

tuner.search(train_padded_sequences, y_train, epochs=10, validation_data=(test_padded_sequences, y_test))
# nlp = model.fit(build_model(),
#                 train_padded_sequences, y_train,
#                 batch_size=batch_size,
#                 epochs=epochs,
#                 validation_data=(test_padded_sequences, y_test),
#                 callbacks=[checkpoint])



Reloading Tuner from /content/hparam_tuning/tuner0.json


In [12]:
tuner.results_summary()

Results summary
Results in /content/hparam_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
embedding_dim: 224
units: 160
dropout_rate: 0.1
learning_rate: 0.0025182289273151
Score: 0.5476999878883362

Trial 4 summary
Hyperparameters:
embedding_dim: 192
units: 128
dropout_rate: 0.0
learning_rate: 0.00022421304870992316
Score: 0.5475999712944031

Trial 2 summary
Hyperparameters:
embedding_dim: 224
units: 192
dropout_rate: 0.1
learning_rate: 0.0005061452043306976
Score: 0.5471000075340271

Trial 1 summary
Hyperparameters:
embedding_dim: 224
units: 256
dropout_rate: 0.1
learning_rate: 0.0008510730813483274
Score: 0.546999990940094

Trial 0 summary
Hyperparameters:
embedding_dim: 224
units: 160
dropout_rate: 0.1
learning_rate: 0.0005664721037007217
Score: 0.5468000173568726


In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint
import keras_tuner

batch_size = 32
epochs = 20

checkpoint = ModelCheckpoint('model_checkpoint.h5', save_best_only=True, monitor='val_loss', mode='min')

model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=224, input_length = max_length))
model.add(LSTM(units=160, return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(units=160, activation='tanh'))
model.add(Dense(units=num_classes, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate =  0.00251822),
              metrics=['accuracy'])

nlp = model.fit(
                train_padded_sequences, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(test_padded_sequences, y_test),
                callbacks=[checkpoint])

model.save('my_model.h5')

Epoch 1/20
623/625 [============================>.] - ETA: 0s - loss: 0.6886 - accuracy: 0.5247

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 10s 12ms/step - loss: 0.6886 - accuracy: 0.5247 - val_loss: 0.6828 - val_accuracy: 0.5236
Epoch 2/20
625/625 [==============================] - 6s 10ms/step - loss: 0.6853 - accuracy: 0.5286 - val_loss: 0.6799 - val_accuracy: 0.5470
Epoch 3/20
625/625 [==============================] - 6s 9ms/step - loss: 0.6980 - accuracy: 0.5203 - val_loss: 0.6843 - val_accuracy: 0.5418
Epoch 4/20
625/625 [==============================] - 6s 9ms/step - loss: 0.6903 - accuracy: 0.5264 - val_loss: 0.6809 - val_accuracy: 0.5444
Epoch 5/20
625/625 [==============================] - 6s 9ms/step - loss: 0.6884 - accuracy: 0.5293 - val_loss: 0.6897 - val_accuracy: 0.5032
Epoch 6/20
625/625 [==============================] - 6s 9ms/step - loss: 0.6909 - accuracy: 0.5222 - val_loss: 0.6808 - val_accuracy: 0.5452
Epoch 7/20
625/625 [==============================] - 6s 10ms/step - loss: 0.6895 - accuracy: 0.5199 - val_loss: 0.6794 - val_accuracy: 0.5246
Epoch 8/20
62

In [43]:
def generate(sen):

  sen = re.sub(html_re, ' ', sen)
  sen = re.sub(url_re, '', sen)
  sen = sen.lower()
  sen = sen.translate(str.maketrans('', '', string.punctuation))

  tokens = word_tokenize(sen)
  tokens = [word for word in tokens if word not in stop_words]
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  tokens = [slang_dict.get(token, token) for token in tokens]

  sequences = tokenizer.texts_to_sequences(sen)
  padded_sequences = pad_sequences(sequences, maxlen=max_length)

  return (np.average((model.predict(padded_sequences) > 0.5).astype(int)) > 0.5).astype(int)